**Niks Pille**

All Tickers

In [9]:
#Getting all tickers in a list
All_Tickers = []
for i in range(4521):
    All_Tickers.append(Only_Common_Stocks['symbol'][i])

Creating 'Stock_With_All_Data'

In [171]:
Stocks_With_All_Data = []
for i in range(4521):
    if len(yf.Ticker(All_Tickers[i]).history(start='2000-01-02', end='2020-01-01')[['Close']]) == 5031:
        Stocks_With_All_Data.append(All_Tickers[i])

- AGM.A: No data found, symbol may be delisted
- ALUS: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- AMHC: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- ARQT: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BATL: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BDTX: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BEAM: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BF.A: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- BH.A: No data found, symbol may be delisted
- BIO.B: No data found for this date range, symbol may be delisted
- BRK.A: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BSBK: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BWL.A: No data found, symbol may be delisted
- CHPM: Data doesn't exist for startDate = 946767600, endDate 

In [186]:
with open('Stocks_With_Data.csv','w',newline='') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    wr.writerow(Stocks_With_All_Data)

# Bachelorproject

Documentation:  
https://addisonlynch.github.io/iexfinance/stable/stocks.html  
https://github.com/ranaroussi/yfinance

## Getting Data

In [190]:
#All packages used
from iexfinance.stocks import Stock
from iexfinance.refdata import get_symbols
import yfinance as yf
import numpy as np
import pandas as pd
import csv
token = 'sk_32c5d1b5f6b044f294391ae88eac3e47'

In [2]:
with open('Stocks_With_Data.csv',newline='') as file:
    reader = csv.reader(file)
    Stocks_With_All_Data = list(reader)

In [3]:
#Russell3000 is used as the market rate since it covers 98% of the American market
Russell_3000 = yf.Ticker('^RUA').history(start='2000-01-02', end='2020-01-01')[['Close']]

In [4]:
#Get a table with everything on the financial markets in USA
Alle_symboler = get_symbols(output_format='pandas', token=token)
#Sorting out all the non common stocks
Only_Common_Stocks = get_symbols()[get_symbols().type == 'cs'].reset_index(drop=True)

In [5]:
#Drawing 300 random stocks for which to be analyzed
Stocks = []
np.random.seed(2020)
for i in range(300):
    Stocks.append(Stocks_With_All_Data[0][np.random.randint(0,len(Stocks_With_All_Data[0]))])

Stocks.sort()

### Adjusting so all data match with no missing values

In [69]:
#DTB3 is taken from https://fred.stlouisfed.org/
DTB3 = pd.read_csv('DTB3.csv', index_col = 0)
#Using Apple as a benchmark
aapl = yf.Ticker('aapl').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Getting dates from Apple
Dates = []
for i in range(len(aapl)):
    if str(aapl.index[i])[:10] in DTB3.index:
        Dates.append(str(aapl.index[i])[:10])
#Sorting out missing values
Three_Month_Treasury_Bill = DTB3.loc[Dates][DTB3.loc[Dates].DTB3 != '.']
#Creating a csv-file with the Three Month Treasury Bill
Three_Month_Treasury_Bill.to_csv('Three_Month_Treasury_Bill.csv')

In [7]:
Risk_Free_Rate = pd.read_csv('Three_Month_Treasury_Bill.csv', index_col=0)

In [8]:
Risk_Free_Rate

,DTB3
DATE,
2000-01-03,5.27
2000-01-04,5.27
2000-01-05,5.28
2000-01-06,5.25
2000-01-07,5.22
...,...
2019-12-24,1.55
2019-12-26,1.55
2019-12-27,1.54


In [185]:
#Make a table with the new dates
Ny = DTB3.loc[Dates]
#Make a new column with strings
Ny['Datoer'] = Dates
#Set the new column as index to make it a callable string
Nyy = Ny.set_index('Datoer')
#A for loop to get all tables for the chosen stocks into a single list, to be used in concatenating
stock = []
for i in Stocks:
    asset = yf.Ticker(i).history(start='2000-01-02', end='2020-01-01')[['Close']]
    asset.columns = [i]
    stock.append(asset)
#Concat all the tables
Concat_Table = pd.concat([i for i in stock], axis=1, join='inner')
#Concat the stocks with the risk free rate
All_Tables = pd.concat([Nyy, Concat_Table.reindex(Nyy.index)],axis=1)
#Save to a csv file
All_Tables.to_csv('All_Stocks.csv')

In [193]:
Everything = pd.read_csv('All_Stocks.csv', index_col=0)

In [194]:
Everything

,DTB3,A,AB,ABC,ADM,AEE,AEM,AES,AES.1,AGM,...,WTM,WVVI,WWD,WWE,WYY,XLNX,XLNX.1,XRAY,XRX,Y
Datoer,,,,,,,,,,,,,,,,,,,,,
2000-01-03,5.270,44.41,7.45,3.00,7.08,11.94,6.14,28.58,28.58,13.74,...,106.47,2.00,3.46,7.33,2.3125,33.44,33.44,6.97,52.59,141.59
2000-01-04,5.270,41.01,7.42,2.79,7.00,11.94,5.98,27.45,27.45,13.39,...,105.57,2.06,3.47,7.01,2.3125,32.68,32.68,6.97,50.15,140.83
2000-01-05,5.280,38.47,7.45,3.01,6.89,12.40,5.93,27.74,27.74,12.68,...,108.63,2.12,3.38,7.27,2.2812,31.96,31.96,7.10,52.73,140.31
2000-01-06,5.250,37.01,7.57,3.24,6.93,12.36,6.03,27.97,27.97,13.17,...,107.83,2.06,3.34,6.81,2.7500,29.19,29.19,7.10,51.64,141.07
2000-01-07,5.220,40.09,7.82,3.66,7.04,12.54,6.19,28.51,28.51,13.52,...,107.15,2.12,3.34,6.84,2.7500,32.50,32.50,7.09,52.86,144.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,1.55,85.12,29.43,85.05,45.72,75.29,61.10,19.62,19.62,84.20,...,1119.00,6.99,118.66,64.92,0.3900,98.06,98.06,57.11,37.12,774.64
2019-12-26,1.55,85.27,29.51,84.68,45.84,75.18,61.07,19.79,19.79,84.74,...,1105.42,7.03,118.37,65.44,0.3800,98.25,98.25,56.53,37.23,776.84
2019-12-27,1.54,85.24,29.24,84.54,45.94,75.84,61.06,19.84,19.84,84.23,...,1098.02,7.03,118.15,64.92,0.4000,97.99,97.99,56.84,37.06,777.77
